# How to use ML Transformer

Quick tutorial for Jian.

In [13]:
from pathlib import Path
import rich
from gnn_tracking.models.graph_construction import MLPCTransformerFromMLChkpt
from gnn_tracking.utils.loading import TrackingDataModule
import gnn_tracking

In [5]:
lightning_home = Path("/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/model_exchange/gc")
assert lightning_home.is_dir()

if gnn_tracking.__version__ >= "23.12.0":
    chkpt_name = "quiet-origami-prawn_compatible.ckpt"
else:
    chkpt_name = "quiet-origami-prawn.ckpt"

ml_chkpt_path = lightning_home / chkpt_name
print(ml_chkpt_path)
assert ml_chkpt_path.is_file()

/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/model_exchange/gc/quiet-origami-prawn_compatible.ckpt


In [6]:
model = MLPCTransformerFromMLChkpt(ml_chkpt_path, original_features=True)

[12:05:37] DEBUG: Getting class MLModule from module gnn_tracking.training.ml
[12:05:38] DEBUG: Loading checkpoint /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/model_exchange/gc/quiet-origami-prawn_compatible.ckpt
[12:05:38] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[12:05:38] DEBUG: Getting class GraphConstructionHingeEmbeddingLoss from module gnn_tracking.metrics.losses.metric_learning
[12:05:38] DEBUG: Getting class GraphConstructionKNNScanner from module gnn_tracking.graph_construction.k_scanner
[12:05:38] DEBUG: Checkpoint loaded. Model ready to go.


In [12]:
rich.print(dict(model.hparams))

{
    'ml': {
        'class_path': 'gnn_tracking.models.graph_construction.GraphConstructionFCNN',
        'init_args': {'in_dim': 14, 'hidden_dim': 256, 'out_dim': 8, 'depth': 6, 'alpha': 0.6}
    },
    'original_features': True,
    'freeze': True
}

As evident fro hparams, output latent space had dimension 8

In [16]:
dm = TrackingDataModule(
    identifier="point_clouds_v8",
    train=dict(
        dirs=[
            "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_1/"
        ],
        sample_size=5,
    ),
    val=dict(
        dirs=[
            "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_9/"
        ],
        stop=5
    ),
)
dm.setup(stage="fit")

[12:09:27] INFO: DataLoader will load 900 graphs (out of 900 available).
[12:09:27] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_1/data21000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_1/data21999_s0.pt
[12:09:27] INFO: DataLoader will load 5 graphs (out of 1000 available).
[12:09:27] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_9/data29004_s0.pt


In [21]:
data = dm.datasets["val"][0].to("cuda")

data.x.shape

torch.Size([59357, 14])

In [22]:
data_with_embedding = model(data)

data.x.shape

torch.Size([59357, 22])

First 8 dimensions are the embedding, remaining 14 the input features.